In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.style.use('ggplot')

# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqluser)

# the below statement is prepended to queries to ensure they select from the right schema
query_schema = 'set search_path to ' + schema_name + ';'

Let's take 20% of all patients, that we will keep for **final** assessment of a method

In [2]:
query = query_schema + """
SELECT DISTINCT subject_id
FROM icustays
"""

df = pd.read_sql_query(query, con)

df_main, df_validation = train_test_split(df, test_size = 0.2, random_state=0)

df_main.to_csv("patient_list_main.csv", index=False)
df_validation.to_csv("patient_list_final_validation.csv", index=False)

df_main["is_validation"] = False
df_validation["is_validation"] = True

df_final = pd.concat((df_main, df_validation))

df_final.head()

/tmp/ipykernel_16637/2510559973.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con)


,subject_id,is_validation
13998,17512,False
13938,17438,False
23048,31303,False
22504,29909,False
7607,9538,False


In [5]:
try:
    cur = con.cursor()
    
    create_table_query = query_schema + """
    CREATE TABLE final_validation_subjects (
        subject_id integer,
        is_validation bool
    );
    """
    
    try:
        cur.execute(create_table_query)
    except psycopg2.errors.DuplicateTable:
        print("Table final_validation_subjects already exists")
        con.rollback()
    cur.close()

    cur = con.cursor()
    insert_query = query_schema + """
    INSERT INTO final_validation_subjects (subject_id, is_validation)
    VALUES (%s, %s);
    """
    
    pars = df_final.values.tolist()
    pars = list(map(tuple, pars))
    cur.executemany(insert_query, pars)
    cur.execute("SELECT * FROM final_validation_subjects LIMIT 10")
    
    con.commit()
    con.close()
    # cur.close()

except psycopg2.Error as e:
    print(f"An error occurred: {e}")
    # Roll back the current transaction
    con.rollback()

Table final_validation_subjects already exists


In [8]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqluser)
query = query_schema + """
SELECT *
FROM final_validation_subjects
"""

df = pd.read_sql_query(query, con)
df.head()

/tmp/ipykernel_16637/1679566991.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con)


,subject_id,is_validation
0,17512,False
1,17438,False
2,31303,False
3,29909,False
4,9538,False
